Environment setup:
```
conda create -n earth2studio python=3.12 -y 
conda activate earth2studio
pip install uv
export UV_CACHE_DIR="/projectnb/eb-general/wade/uv_cache"
uv pip install "earth2studio @ git+https://github.com/NVIDIA/earth2studio.git@0.10.0"
uv pip install "earth2studio[fcn]"
uv pip install numpy matplotlib pandas xarray cartopy cmocean tqdm
uv pip install "makani @ git+https://github.com/NVIDIA/modulus-makani.git@28f38e3e929ed1303476518552c64673bbd6f722"
uv pip install earth2studio[sfno]
```


# Running inference with SFNO checkpoints

In [1]:
import os
import subprocess
from dotenv import load_dotenv

from earth2studio.io import ZarrBackend

# from earth2studio.run import deterministic
# from earth2studio.models.px import SFNO
### Updated source code for checkpoint selection and 
### specific variable saving functionality:
from deterministic_update import deterministic
from SFNO_update import SFNO

import earth2studio.data as data
from earth2studio.models.auto import Package
from utils import filename_to_year, datetime_range, open_hdf5 # these aren't used in this script currently

from datetime import datetime, timedelta
import json
import xarray as xr
from typing import List
import shutil
import sys
import gc
import numpy as np
import time

import torch

/projectnb/eb-general/wade/.conda/envs/earth2studio/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/projectnb/eb-general/wade/.conda/envs/earth2studio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Should be using earth2studio version 0.10.0 and makani version 0.2.0.

In [2]:
# print versions of makani and earth2studio
import earth2studio
print(f"earth2studio version: {earth2studio.__version__}")
import makani
print(f"makani version: {makani.__version__}")


earth2studio version: 0.10.0rc0
makani version: 0.2.0


In [3]:
# Check if CUDA (GPU support) is available
is_available = torch.cuda.is_available()
print(f"Is CUDA available? {is_available}")

if is_available:
    # Get the number of available GPUs
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs available: {gpu_count}")

    # Get the ID of the current GPU
    current_gpu = torch.cuda.current_device()
    print(f"Current GPU ID: {current_gpu}")

    # Get the name of the current GPU
    gpu_name = torch.cuda.get_device_name(current_gpu)
    print(f"Current GPU Name: {gpu_name}")

    print(f"Memory (VRAM):      {torch.cuda.get_device_properties(current_gpu).total_memory / 1e9:.2f} GB")
else:
    print("CUDA is not available. Running on CPU.")

time_start = time.time()


Is CUDA available? True
Number of GPUs available: 1
Current GPU ID: 0
Current GPU Name: NVIDIA RTX A6000
Memory (VRAM):      47.70 GB


In [4]:
############# CONFIG FOR INFERENCE RUN #############

start_datetime = "2022-12-22T00:00:00" #"2019-09-03T00:00:00" #"2022-09-24T00:00:00" # "2021_09_20T00:00:00" # 
variables_to_select = ['tcwv', 'u10m', 'v10m'] #['msl'] #Only save selected variables - it slows down inference SIGNIFICANTLY to save all 74 variables
experiment_number = 1 # which experiment directory to output to
n_steps = 20  # number of 6hr steps to forecast
epochs_to_run = np.arange(1,90,2) # List or array of epochs/checkpoint numbers to run inference on

# boring =  False
ema = False

# Create the inference name based on the start datetime and number of steps
inference_name = datetime.fromisoformat(start_datetime).strftime("%Y_%m_%dT%H")+'_nsteps'+str(n_steps)
data_create_fp = "/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/init_files/Initialize_"+inference_name+".nc" # "/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_"+inference_name+".nc"

# Calculate the final datetime based from the start datetime and number of steps
final_datetime = (datetime.fromisoformat(start_datetime) + timedelta(hours = int(n_steps*6))).isoformat() 

# Directories
results_out_dir = f"/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment{str(experiment_number)}/{final_datetime[:10].replace('-', '_')}/"

#################################################

In [5]:
data_create_fp, final_datetime, results_out_dir

('/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/init_files/Initialize_2022_12_22T00_nsteps20.nc',
 '2022-12-27T00:00:00',
 '/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/')

In [ ]:
if os.path.exists(data_create_fp):
    print(f"Data already preprocessed: {data_create_fp}")
else:
    sys.exit(f"Data not found use Create_Initial_Data.ipynb to create: {data_create_fp}")

# make this xarray into a dataarray file for earth2studio
initial_data = data.DataArrayFile(data_create_fp)

fine_tuning_start_epoch = 71 # the epoch where fine-tuning starts (important for correctly accessing the checkpoints)

time_1 = time.time()
print(f"Data loaded in {time_1 - time_start:.2f} seconds")

for n_epoch in epochs_to_run: 
    time_2 = time.time()

    if ema:
        results_out_fp = results_out_dir+f"EMA_Checkpoint{n_epoch}_{inference_name}.nc"
    else:
        results_out_fp =  results_out_dir+"Checkpoint"+str(n_epoch)+"_"+inference_name+'.nc' 
    
    # Check if the results file already exists
    if os.path.exists(results_out_fp):
        print(f"Results file {results_out_fp} already exists. Skipping to next epoch.")
        continue  # Skip the rest of the loop and go to the next iteration
    else:
        os.makedirs(os.path.dirname(results_out_fp), exist_ok=True)

        load_dotenv()

        if n_epoch < fine_tuning_start_epoch: # pre-fine-tuning phase epochs are numbered 1-70
            src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"
            checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch)+'.tar'
        else:
            src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/"
            n_epoch_multistep2 = n_epoch - (fine_tuning_start_epoch - 1) # fine-tuning phase epochs are numbered from 1-20
            checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch_multistep2)+'.tar'

        print(f"Loading model from {src_dir} with checkpoint {checkpoint_name}")
        # Load the model package from storage
        model_package = Package(src_dir, cache = False)
        model = SFNO.load_model(model_package, 
        checkpoint_name = checkpoint_name, EMA = ema
        )

        # Create the IO handler, store in memory
        io = ZarrBackend()
        
        with torch.no_grad():
            # run inference
            io = deterministic([start_datetime], n_steps, model, initial_data, io, 
            variables_list=variables_to_select
            )

        print(io.root.tree())

        # save results to netcdf
        # Open the Zarr group from the in-memory store using xarray
        ds = xr.open_zarr(io.root.store)
        
        # SANITY CHECKING...
        print("Dataset times:", ds["time"].values)
        print("Dataset dimensions:", {dim: ds.dims[dim] for dim in ds.dims})
        print("Lead times", ds["lead_time"].values)

        # Convert the 'time' coordinate in ds to datetime64 format
        ds["time"] = ds["time"].astype("datetime64[ns]")

        # Convert lead_time from nanoseconds to timedelta64[ns]
        base_time = ds["time"].values  # shape (n_time,)
        lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")  # shape (n_lead_time,)
        # Broadcast to 2D: (time, lead_time)
        valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten() 
        # Drop the old lead_time coordinate
        ds = ds.drop_vars("lead_time")

        # Assume ds has dimensions (time, lead_time, lat, lon) and only one time
        initial_time = str(ds["time"].values[0])  # Save the initial time as a string
        # Remove the time dimension by selecting the first (and only) time
        ds = ds.isel(time=0).drop_vars("time")
        # Add the initial time as a global attribute
        ds.attrs["initial_time"] = initial_time

        # Create valid_time by adding lead_timedelta to base_time
        ds = ds.rename({"lead_time": "valid_time"})
        # Assign valid_time as a coordinate
        ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

        lead_times_to_save = np.arange(1,9,2) # in days
        lead_times_to_save = lead_times_to_save[lead_times_to_save <= n_steps//4] # only keep lead times that are within the n_steps range

        # Calculate the specific timestamps for these days
        start_dt = datetime.fromisoformat(start_datetime)
        target_timestamps = []
        for d in lead_times_to_save:
            ts = start_dt + timedelta(days=d)
            target_timestamps.append(np.datetime64(ts))

        print(f"Attempting to save forecasts for days: {lead_times_to_save}")

        available_times = ds["valid_time"].values
        times_to_save = [t for t in target_timestamps if t in available_times] # overlap between target lead times and available times

        if len(times_to_save) > 0:
            ds_subset = ds.sel(valid_time=times_to_save)
            ds_subset = ds_subset[variables_to_select]
            ds_subset.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
            print(f"Results saved to {results_out_fp} with {len(times_to_save)} timesteps.")
        else:
            print(f"ERROR: None of the target odd days were found in the output. available_times: {available_times}")

        # # only save the final time step
        # if np.datetime64(final_datetime) in ds["valid_time"].values:
        #     ds = ds.sel(valid_time=[final_datetime])
        #     ds = ds[variables_to_select]
        #     ds.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
        #     print(f"Results saved to {results_out_fp}")
        # else:
        #     print(f"ERROR: final_datetime {final_datetime} not found in ds['valid_time']. No file saved.")

        #some cleanup
        torch.cuda.empty_cache()
        del model_package
        del model
        del io
        del ds
        gc.collect()
        time_3 = time.time()
        print(f"Epoch {n_epoch} done: {time_3 - time_2:.2f} seconds")


Data already preprocessed: /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/init_files/Initialize_2022_12_22T00_nsteps20.nc
Data loaded in 5.40 seconds
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/ with checkpoint ckpt_mp0_epoch1.tar
2026-01-05 14:27:21.286 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 14:27:21.314 | INFO     | deterministic_update:deterministic:68 - Inference device: cpu
2026-01-05 14:27:22.057 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 14:27:22.300 | INFO     | deterministic_update:deterministic:122 - Inference starting!


# Code with GPU utilization and timing monitoring

In [7]:
if os.path.exists(data_create_fp):
    print(f"Data already preprocessed: {data_create_fp}")
else:
    sys.exit(f"Data not found use Create_Initial_Data.ipynb to create: {data_create_fp}")

# make this xarray into a dataarray file for earth2studio
initial_data = data.DataArrayFile(data_create_fp)

fine_tuning_start_epoch = 71 

time_1 = time.time()
print(f"Data loaded in {time_1 - time_start:.2f} seconds")

for n_epoch in epochs_to_run: 
    time_2 = time.time()
    
    # --- MONITORING START: Reset peak memory tracker for this epoch ---
    torch.cuda.reset_peak_memory_stats()
    # ----------------------------------------------------------------

    if ema:
        results_out_fp = results_out_dir+f"EMA_Checkpoint{n_epoch}_{inference_name}.nc"
    else:
        results_out_fp =  results_out_dir+"Checkpoint"+str(n_epoch)+"_"+inference_name+'.nc' 
    
    # Check if the results file already exists
    if os.path.exists(results_out_fp):
        print(f"Results file {results_out_fp} already exists. Skipping to next epoch.")
        continue 
    else:
        os.makedirs(os.path.dirname(results_out_fp), exist_ok=True)

        load_dotenv()

        if n_epoch < fine_tuning_start_epoch: 
            src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"
            checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch)+'.tar'
        else:
            src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/"
            n_epoch_multistep2 = n_epoch - (fine_tuning_start_epoch - 1) 
            checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch_multistep2)+'.tar'

        print(f"Loading model from {src_dir} with checkpoint {checkpoint_name}")
        model_package = Package(src_dir, cache = False)
        model = SFNO.load_model(model_package, 
        checkpoint_name = checkpoint_name, EMA = ema
        )

        io = ZarrBackend()
        
        with torch.no_grad():
            io = deterministic([start_datetime], n_steps, model, initial_data, io, 
            variables_list=variables_to_select
            )

        # --- MONITORING: Capture GPU stats immediately after inference ---
        peak_mem = torch.cuda.max_memory_allocated() / 1e9 # Convert to GB
        try:
            # Quick snapshot of GPU utilization via nvidia-smi
            gpu_stats = subprocess.check_output(
                ["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"], 
                encoding='utf-8'
            ).strip()
            gpu_util = f"{gpu_stats}%"
        except:
            gpu_util = "N/A"
        # ---------------------------------------------------------------

        # save results to netcdf
        ds = xr.open_zarr(io.root.store)
        
        ds["time"] = ds["time"].astype("datetime64[ns]")

        base_time = ds["time"].values  
        lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")  
        valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten() 
        ds = ds.drop_vars("lead_time")

        initial_time = str(ds["time"].values[0]) 
        ds = ds.isel(time=0).drop_vars("time")
        ds.attrs["initial_time"] = initial_time

        ds = ds.rename({"lead_time": "valid_time"})
        ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

        lead_times_to_save = np.arange(1,9,2) # in days
        lead_times_to_save = lead_times_to_save[lead_times_to_save <= n_steps//4] 

        start_dt = datetime.fromisoformat(start_datetime)
        target_timestamps = []
        for d in lead_times_to_save:
            ts = start_dt + timedelta(days=int(d))
            target_timestamps.append(np.datetime64(ts))

        available_times = ds["valid_time"].values
        times_to_save = [t for t in target_timestamps if t in available_times] 

        if len(times_to_save) > 0:
            ds_subset = ds.sel(valid_time=times_to_save)
            ds_subset = ds_subset[variables_to_select]
            ds_subset.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
            print(f"Results saved to {results_out_fp} with {len(times_to_save)} timesteps.")
        else:
            print(f"ERROR: None of the target odd days were found. available_times: {available_times}")

        # Cleanup
        torch.cuda.empty_cache()
        del model_package
        del model
        del io
        del ds
        gc.collect()
        
        # --- MONITORING: Final timing prints ---
        time_3 = time.time()
        epoch_dur = time_3 - time_2
        total_elapsed = time_3 - time_start
        print(f"Epoch {n_epoch} Stats:")
        print(f"  Duration: {epoch_dur:.2f}s | Total Elapsed: {total_elapsed/60:.2f} min")
        print(f"  Peak VRAM: {peak_mem:.2f} GB | GPU Util Snapshot: {gpu_util}")
        print("-" * 50)
        # ---------------------------------------

Data already preprocessed: /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/init_files/Initialize_2022_12_22T00_nsteps20.nc
Data loaded in 464.76 seconds
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/ with checkpoint ckpt_mp0_epoch1.tar
2026-01-05 14:46:16.397 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 14:46:16.397 | INFO     | deterministic_update:deterministic:68 - Inference device: cuda
2026-01-05 14:46:16.688 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 14:46:16.723 | INFO     | deterministic_update:deterministic:122 - Inference starting!


Running inference: 100%|██████████| 21/21 [00:15<00:00,  1.32it/s]


2026-01-05 14:46:32.608 | SUCCESS  | deterministic_update:deterministic:146 - Inference complete
Results saved to /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/Checkpoint1_2022_12_22T00_nsteps20.nc with 3 timesteps.
Epoch 1 Stats:
  Duration: 26.27s | Total Elapsed: 8.18 min
  Peak VRAM: 16.25 GB | GPU Util Snapshot: 0
0
0
0
0
0
0
0%
--------------------------------------------------
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/ with checkpoint ckpt_mp0_epoch3.tar
2026-01-05 14:47:11.388 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 14:47:11.389 | INFO     | deterministic_update:deterministic:68 - Inference device: cuda
2026-01-05 14:47:11.710 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 14:47:11.750 | INFO     | deterministic_update:determin

Running inference: 100%|██████████| 21/21 [00:15<00:00,  1.32it/s]


2026-01-05 14:47:27.692 | SUCCESS  | deterministic_update:deterministic:146 - Inference complete
Results saved to /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/Checkpoint3_2022_12_22T00_nsteps20.nc with 3 timesteps.
Epoch 3 Stats:
  Duration: 53.41s | Total Elapsed: 9.07 min
  Peak VRAM: 16.25 GB | GPU Util Snapshot: 100
0
0
0
0
0
0
0%
--------------------------------------------------
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/ with checkpoint ckpt_mp0_epoch5.tar
2026-01-05 14:48:04.863 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 14:48:04.863 | INFO     | deterministic_update:deterministic:68 - Inference device: cuda
2026-01-05 14:48:05.236 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 14:48:05.276 | INFO     | deterministic_update:determ

...

2026-01-05 15:22:53.549 | SUCCESS  | deterministic_update:deterministic:146 - Inference complete
Results saved to /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/Checkpoint85_2022_12_22T00_nsteps20.nc with 3 timesteps.
Epoch 85 Stats:
  Duration: 36.71s | Total Elapsed: 44.51 min
  Peak VRAM: 16.25 GB | GPU Util Snapshot: 0
0
0
0
0
0
0
0%
--------------------------------------------------
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/ with checkpoint ckpt_mp0_epoch17.tar
2026-01-05 15:23:12.874 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 15:23:12.874 | INFO     | deterministic_update:deterministic:68 - Inference device: cuda
2026-01-05 15:23:13.293 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 15:23:13.333 | INFO     | dete

Running inference: 100%|██████████| 21/21 [00:15<00:00,  1.32it/s]


2026-01-05 15:23:29.280 | SUCCESS  | deterministic_update:deterministic:146 - Inference complete
Results saved to /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/Checkpoint87_2022_12_22T00_nsteps20.nc with 3 timesteps.
Epoch 87 Stats:
  Duration: 35.74s | Total Elapsed: 45.10 min
  Peak VRAM: 16.25 GB | GPU Util Snapshot: 0
0
0
0
0
0
0
0%
--------------------------------------------------
Loading model from /projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/ with checkpoint ckpt_mp0_epoch19.tar
2026-01-05 15:23:49.317 | INFO     | deterministic_update:deterministic:61 - Running simple workflow!
2026-01-05 15:23:49.317 | INFO     | deterministic_update:deterministic:68 - Inference device: cuda
2026-01-05 15:23:49.760 | SUCCESS  | deterministic_update:deterministic:92 - Fetched data from DataArrayFile
2026-01-05 15:23:49.800 | INFO     | dete

Running inference: 100%|██████████| 21/21 [00:15<00:00,  1.32it/s]


2026-01-05 15:24:05.745 | SUCCESS  | deterministic_update:deterministic:146 - Inference complete
Results saved to /projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment1/2022_12_27/Checkpoint89_2022_12_22T00_nsteps20.nc with 3 timesteps.
Epoch 89 Stats:
  Duration: 36.45s | Total Elapsed: 45.71 min
  Peak VRAM: 16.25 GB | GPU Util Snapshot: 100
0
0
0
0
0
0
0%
--------------------------------------------------


In [3]:
# Compare the output of my forecast to Becca's saved forecast
my_forecast ='/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment0/2022_09_29/Checkpoint70_2022_09_24T00_nsteps20.nc' 
beccas_forecast = '/projectnb/eb-general/wade/sfno/inference_runs/Ian/leadtime_fivedays/Checkpoint70_2022_09_24T00_nsteps20.nc'
my_ds = xr.open_dataset(my_forecast)
beccas_ds = xr.open_dataset(beccas_forecast)
print(my_ds)
print(beccas_ds)

<xarray.Dataset> Size: 12MB
Dimensions:     (valid_time: 1, lat: 721, lon: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2022-09-29
  * lat         (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon         (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    msl         (valid_time, lat, lon) float32 4MB ...
    u10m        (valid_time, lat, lon) float32 4MB ...
    v10m        (valid_time, lat, lon) float32 4MB ...
Attributes:
    initial_time:  2022-09-24T00:00:00.000000000
<xarray.Dataset> Size: 12MB
Dimensions:     (valid_time: 1, lat: 721, lon: 1440)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 8B 2022-09-29
  * lat         (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon         (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    msl         (valid_time, lat, lon) float32 4MB ...
    u10m        (valid_time, lat, lon) float32 4MB ...
    v10m

To check what epoch a checkpoint is (e.g. if epoch number is not in the filepath):

In [38]:
import torch
import os

dir='/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/training_checkpoints/' # step 1 of training (epochs 1-70)
dir2='/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/training_checkpoints/' # step 2 of training (epochs 71-90

# List of files to check
files_to_check = [dir + "best_ckpt_mp0.tar", 
                  dir + "ckpt_mp0.tar",
                  dir2 +  "best_ckpt_mp0.tar",
                    dir2 + "ckpt_mp0.tar"
                    ]
                
for filename in files_to_check:
    # Load the checkpoint
    # map_location='cpu' allows you to inspect this even without a GPU
    # weights_only=False allows loading the full dictionary structure
    checkpoint = torch.load(filename, map_location='cpu', weights_only=False)
    epoch = checkpoint.get('epoch', 'N/A')
    
    print(f"{filename:<25}")
    print(f'epoch: {str(epoch)}')
    print()

/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/training_checkpoints/best_ckpt_mp0.tar
epoch: 70

/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/training_checkpoints/ckpt_mp0.tar
epoch: 70

/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/training_checkpoints/best_ckpt_mp0.tar
epoch: 19

/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/training_checkpoints/ckpt_mp0.tar
epoch: 20



the above output shows that the best_ckpt is the best ckpt *for each phase*, and ckpt_mp0 is the *final* ckpt for that phase
- checkpoint numbering resets for each phase so phase 2 is numbered 1-20


# Original script:

In [ ]:
import os
import subprocess
from dotenv import load_dotenv

from earth2studio.io import ZarrBackend
from SFNO_update import SFNO
import earth2studio.data as data
from earth2studio.models.auto import Package
from utils import filename_to_year, datetime_range, open_hdf5
from deterministic_update import deterministic

from datetime import datetime, timedelta
import json
import xarray as xr
from typing import List
import shutil
import sys
import gc
import numpy as np
import time

import torch

# Check if CUDA (GPU support) is available
is_available = torch.cuda.is_available()
print(f"Is CUDA available? {is_available}")

if is_available:
    # Get the number of available GPUs
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs available: {gpu_count}")

    # Get the ID of the current GPU
    current_gpu = torch.cuda.current_device()
    print(f"Current GPU ID: {current_gpu}")

    # Get the name of the current GPU
    gpu_name = torch.cuda.get_device_name(current_gpu)
    print(f"Current GPU Name: {gpu_name}")

    print(f"Memory (VRAM):      {torch.cuda.get_device_properties(current_gpu).total_memory / 1e9:.2f} GB")
else:
    print("CUDA is not available. Running on CPU.")

time_start = time.time()

############# Double check these before running the script #############
#select start datetime and n_steps, each n_step = 6hrs
start_datetime = "2021-09-20T00:00:00" # "2021_09_20T00:00:00"
variables_to_select = ['tcwv'] #Only save selected variables - it slows down inference SIGNIFICANTLY to save all 74 variables
experiment_number = 0
n_steps = 20  #number of 6hr steps to forecast

boring = False
ema = False

# Create the inference name based on the start datetime and number of steps
inference_name = datetime.fromisoformat(start_datetime).strftime("%Y_%m_%dT%H")+'_nsteps'+str(n_steps)
data_create_fp = "/projectnb/eb-general/wade/sfno/inference_runs/Ian/Initialize_data/Initialize_"+inference_name+".nc"

# Calculate the final datetime based from the start datetime and number of steps
final_datetime = (datetime.fromisoformat(start_datetime) + timedelta(hours = int(n_steps*6))).isoformat() 

# Directories
results_out_dir = f"/projectnb/eb-general/wade/sfno/inference_runs/sandbox/Experiment{str(experiment_number)}/{final_datetime[:10].replace('-', '_')}/"

############# Double check these before running the script #############


if os.path.exists(data_create_fp):
    print(f"Data already preprocessed: {data_create_fp}")
else:
    sys.exit(f"Data not found use Create_Initial_Data.ipynb to create: {data_create_fp}")

#make this xarray into a dataarray file for earth2studio
initial_data = data.DataArrayFile(data_create_fp)

time_1 = time.time()
print(f"Data loaded in {time_1 - time_start:.2f} seconds")


for n_epoch in np.arange(1,3): #70,1):
    time_2 = time.time()
    # if boring:
    #     # Create the final datetime string in the desired format
    #     
    # else:# Create the final datetime string in the desired format

    if ema:
        results_out_fp = results_out_dir+f"EMA_Checkpoint{n_epoch}_{inference_name}.nc"
    else:
        results_out_fp = results_out_fp = results_out_dir+"/Checkpoint"+str(n_epoch)+"_"+inference_name+'.nc' 
    
    # Check if the results file already exists
    if os.path.exists(results_out_fp):
        print(f"Results file {results_out_fp} already exists. Skipping to next epoch.")
        continue  # Skip the rest of the loop and go to the next iteration
    else:
        os.makedirs(os.path.dirname(results_out_fp), exist_ok=True)

        load_dotenv()  

        # Make temporary folder with all the metadata in it.
        src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"
        # Load the model package from storage
        model_package = Package(src_dir, cache = False)
        model = SFNO.load_model(model_package, checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch)+'.tar', EMA = ema)

        # Create the IO handler, store in memory
        io = ZarrBackend()
        
        with torch.no_grad():
            # run inference
            io = deterministic([start_datetime], n_steps, model, initial_data, io, variables_list=variables_to_select)

        print(io.root.tree())


        # save results to netcdf
        # Open the Zarr group from the in-memory store using xarray
        ds = xr.open_zarr(io.root.store)

        # Convert the 'time' coordinate in ds to datetime64 format
        ds["time"] = ds["time"].astype("datetime64[ns]")

        # Convert lead_time from nanoseconds to timedelta64[ns]
        base_time = ds["time"].values  # shape (n_time,)
        lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")  # shape (n_lead_time,)
        # Broadcast to 2D: (time, lead_time)
        valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten() 
        # Drop the old lead_time coordinate
        ds = ds.drop_vars("lead_time")

        # Assume ds has dimensions (time, lead_time, lat, lon) and only one time
        initial_time = str(ds["time"].values[0])  # Save the initial time as a string
        # Remove the time dimension by selecting the first (and only) time
        ds = ds.isel(time=0).drop_vars("time")
        # Add the initial time as a global attribute
        ds.attrs["initial_time"] = initial_time

        # Create valid_time by adding lead_timedelta to base_time
        ds = ds.rename({"lead_time": "valid_time"})
        # Assign valid_time as a coordinate
        ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

        # only save the final time step
        if np.datetime64(final_datetime) in ds["valid_time"].values:
            ds = ds.sel(valid_time=[final_datetime])
            ds = ds[variables_to_select]
            ds.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
            print(f"Results saved to {results_out_fp}")
        else:
            print(f"ERROR: final_datetime {final_datetime} not found in ds['valid_time']. No file saved.")


        #some cleanup
        torch.cuda.empty_cache()
        del model_package
        del model
        del io
        del ds
        gc.collect()
        time_3 = time.time()
        print(f"Epoch {n_epoch} done: {time_3 - time_2:.2f} seconds")



#     for n_epoch in np.arange(36,71,1):
#         time_2 = time.time()
#         if boring:
#             # Create the final datetime string in the desired format
#             results_out_fp = "/barnes-engr-scratch2/C837824079/Experiment"+str(experiment_number)+"/Forecasts_Boring/"+final_datetime[:10].replace("-", "_")+"/Checkpoint"+str(n_epoch)+"_"+inference_name+'.nc'
#         else:# Create the final datetime string in the desired format
#             if ema:
#                 results_out_fp = f"/barnes-engr-scratch2/C837824079/Experiment{str(experiment_number)}/Forecast/EMA_9/Checkpoint{n_epoch}_{inference_name}.nc"         
#             else:
#                 results_out_fp = "/projectnb/eb-general/rbaiman/SFNO/Example_Inference/Example_Forecast/Checkpoint"+str(n_epoch)+"_"+inference_name+'.nc'

#         # Check if the results file already exists
#         if os.path.exists(results_out_fp):
#             print(f"Results file {results_out_fp} already exists. Skipping to next epoch.")
#             continue  # Skip the rest of the loop and go to the next iteration
#         else:
#             os.makedirs(os.path.dirname(results_out_fp), exist_ok=True)

#             load_dotenv()  

#             # Make temporary folder with all the metadata in it.
#             src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999/"

#             # Load the model package from storage
#             model_package = Package(src_dir, cache = False)
#             model = SFNO.load_model(model_package, checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch)+'.tar', EMA = ema)

#             # Create the IO handler, store in memory
#             io = ZarrBackend()
            
#             with torch.no_grad():
#                 # run inference
#                 io = deterministic([start_datetime], n_steps, model, initial_data, io, variables_list=variables_to_select)

#             print(io.root.tree())


#             # save results to netcdf
#             # Open the Zarr group from the in-memory store using xarray
#             ds = xr.open_zarr(io.root.store)

#             # Convert the 'time' coordinate in ds to datetime64 format
#             ds["time"] = ds["time"].astype("datetime64[ns]")

#             # Convert lead_time from nanoseconds to timedelta64[ns]
#             base_time = ds["time"].values  # shape (n_time,)
#             lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")  # shape (n_lead_time,)
#             # Broadcast to 2D: (time, lead_time)
#             valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten() 
#             # Drop the old lead_time coordinate
#             ds = ds.drop_vars("lead_time")

#             # Assume ds has dimensions (time, lead_time, lat, lon) and only one time
#             initial_time = str(ds["time"].values[0])  # Save the initial time as a string
#             # Remove the time dimension by selecting the first (and only) time
#             ds = ds.isel(time=0).drop_vars("time")
#             # Add the initial time as a global attribute
#             ds.attrs["initial_time"] = initial_time

#             # Create valid_time by adding lead_timedelta to base_time
#             ds = ds.rename({"lead_time": "valid_time"})
#             # Assign valid_time as a coordinate
#             ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

#             # only save the final time step
#             if np.datetime64(final_datetime) in ds["valid_time"].values:
#                 ds = ds.sel(valid_time=[final_datetime])
#                 ds = ds[variables_to_select]
#                 ds.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
#                 print(f"Results saved to {results_out_fp}")
#             else:
#                 print(f"ERROR: final_datetime {final_datetime} not found in ds['valid_time']. No file saved.")


#             #some cleanup
#             torch.cuda.empty_cache()
#             del model_package
#             del model
#             del io
#             del ds
#             gc.collect()
#             time_3 = time.time()
#             print(f"Epoch {n_epoch} done: {time_3 - time_2:.2f} seconds")


# for n_epoch in np.arange(1,21,1):
#     time_2 = time.time()
#     # Create the final datetime string in the desired format
#     if boring:
#         # Create the final datetime string in the desired format
#         results_out_fp = "/barnes-engr-scratch2/C837824079/Experiment"+str(experiment_number)+"/Forecasts_Boring/"+final_datetime[:10].replace("-", "_")+"/Checkpoint"+str(n_epoch+70)+"_"+inference_name+'.nc'
#     else:# Create the final datetime string in the desired format
#         if ema:
#             results_out_fp = f"/barnes-engr-scratch2/C837824079/Experiment{str(experiment_number)}/Forecast/EMA_9/Checkpoint{n_epoch+70}_{inference_name}.nc"
#         else:
#             results_out_fp = "/projectnb/eb-general/rbaiman/SFNO/Example_Inference/Example_Forecast/Checkpoint"+str(n_epoch+70)+"_"+inference_name+'.nc'

    
#     # Check if the results file already exists
#     if os.path.exists(results_out_fp):
#         print(f"Results file {results_out_fp} already exists. Skipping to next epoch.")
#         continue  # Skip the rest of the loop and go to the next iteration
#     else:
#         os.makedirs(os.path.dirname(results_out_fp), exist_ok=True)

#         load_dotenv()  

#         # Make temporary folder with all the metadata in it.
#         src_dir = "/projectnb/eb-general/shared_data/data/processed/FourCastNet_sfno/Checkpoints_SFNO/multistep_sfno_linear_74chq_sc3_layers8_edim384_dt6h_wstgl2/v0.1.0-seed999-multistep2/"

#         # Load the model package from storage
#         model_package = Package(src_dir, cache = False)
#         model = SFNO.load_model(model_package, checkpoint_name = 'ckpt_mp0_epoch'+str(n_epoch)+'.tar', EMA = ema)

#         # Create the IO handler, store in memory
#         io = ZarrBackend()

#         print(f"Running inference for {inference_name}")
#         with torch.no_grad():
#             # run inference
#             io = deterministic([start_datetime], n_steps, model, initial_data, io, variables_list=variables_to_select)

#         # print(io.root.tree())

#         # save results to netcdf
#         # Open the Zarr group from the in-memory store using xarray
#         ds = xr.open_zarr(io.root.store)

#         # Convert the 'time' coordinate in ds to datetime64 format
#         ds["time"] = ds["time"].astype("datetime64[ns]")

#         # Convert lead_time from nanoseconds to timedelta64[ns]
#         base_time = ds["time"].values  # shape (n_time,)
#         lead_timedelta = ds["lead_time"].values.astype("timedelta64[ns]")  # shape (n_lead_time,)
#         # Broadcast to 2D: (time, lead_time)
#         valid_timesteps = (base_time[:, None] + lead_timedelta[None, :]).flatten() 
#         # Drop the old lead_time coordinate
#         ds = ds.drop_vars("lead_time")

#         # Assume ds has dimensions (time, lead_time, lat, lon) and only one time
#         initial_time = str(ds["time"].values[0])  # Save the initial time as a string
#         # Remove the time dimension by selecting the first (and only) time
#         ds = ds.isel(time=0).drop_vars("time")
#         # Add the initial time as a global attribute
#         ds.attrs["initial_time"] = initial_time

#         # Create valid_time by adding lead_timedelta to base_time
#         ds = ds.rename({"lead_time": "valid_time"})
#         # Assign valid_time as a coordinate
#         ds = ds.assign_coords(valid_time=(("valid_time",), valid_timesteps))

#         # only save the final time step
#         if np.datetime64(final_datetime) in ds["valid_time"].values:
#             ds = ds.sel(valid_time=[final_datetime])
#             ds = ds[variables_to_select]
#             ds.to_netcdf(results_out_fp, mode="w", format="NETCDF4")
#             print(f"Results saved to {results_out_fp}")
#         else:
#             print(f"ERROR: final_datetime {final_datetime} not found in ds['valid_time']. No file saved.")


#         #some cleanup
#         torch.cuda.empty_cache()
#         del model_package
#         del model
#         del io
#         del ds
#         gc.collect()
#         time_3 = time.time()
#         print(f"Epoch {n_epoch+70} done: {time_3 - time_2:.2f} seconds")

